<a href="https://colab.research.google.com/github/l-Monarch-l/Laborat/blob/main/GPT_FOR_LAB_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import random
from collections import defaultdict

In [46]:
def stable_softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

In [53]:
# 2. Генерация SCP датасета на русском
scp_texts = [
    "SCP-173 - это статуя, которая двигается, когда за ней не наблюдают, и ломает шеи",
    "SCP-682 - огромное рептилоидное существо, которое крайне сложно уничтожить",
    "SCP-096 - гуманоид, который впадает в ярость, если кто-то увидит его лицо",
    "SCP-106 - старик, который может проходить сквозь стены и утаскивает жертв в свой карманный мир",
    "SCP-049 - врач-чумной доктор, который считает, что может вылечить болезнь",
    "SCP-999 - дружелюбный оранжевый слизень, который делает людей счастливыми",
    "SCP-035 - маска, которая принимает контроль над тем, кто её надевает",
    "SCP-939 - хищные существа, имитирующие человеческие голоса",
    "SCP-076 - Абель, воин с мечом, который постоянно возрождается",
    "SCP-087 - лестница, ведущая в бесконечную темноту",
    "SCP-294 - кофейный аппарат, который может налить любую жидкость",
    "SCP-914 - машина для переработки, которая модифицирует предметы",
    "SCP-1048 - плюшевый медвежонок, который создаёт своих копий",
    "SCP-1128 - паразитическая амфибия, обитающая в водоёмах",
    "SCP-1504 - демон-вещатель, который манипулирует информацией",
    "SCP-1983 - тюрьма, которая удерживает бестелесные сущности",
    "SCP-3008 - бесконечный магазин IKEA с агрессивными сотрудниками",
    "SCP-3199 - мутировавшие существа, ненавидящие человечество",
    "SCP-3301 - интернет-сущность, собирающая информацию",
    "SCP-354 - кровавый пузырь, порождающий опасных существ",
    "SCP-1845 - кукла, которая заставляет людей исчезать",
    "SCP-2399 - повреждённый космический корабль, который чинит себя",
    "SCP-261 - торговый автомат, продающий странные продукты",
    "SCP-426 - тостер, который заставляет людей говорить о себе в первом лице",
    "SCP-529 - кошка-полукровка с отсутствующей задней частью тела",
    "SCP-966 - невидимые существа, мешающие спать",
    "SCP-1048 - плюшевый медведь, который размножается",
    "SCP-1471 - приложение, которое показывает вам существо",
    "SCP-1875 - куклы, которые предсказывают будущее",
    "SCP-2521 - существо, которое похищает тех, кто о нём говорит"
]

In [60]:
# 3. Создание словаря
word_to_idx = defaultdict(lambda: len(word_to_idx))
idx_to_word = {}
text_sequences = []
for text in scp_texts:
    words = text.lower().split()
    sequence = [word_to_idx[word] for word in words]
    text_sequences.append(sequence)
    for word in words:
        idx_to_word[word_to_idx[word]] = word

vocab_size = len(word_to_idx)
print(f"Размер словаря: {vocab_size}")
print(f"Примеры слов в словаре: {list(idx_to_word.items())[:10]}")

Размер словаря: 187
Примеры слов в словаре: [(0, 'scp-173'), (1, '-'), (2, 'это'), (3, 'статуя,'), (4, 'которая'), (5, 'двигается,'), (6, 'когда'), (7, 'за'), (8, 'ней'), (9, 'не')]


In [61]:
# 4. Подготовка данных для обучения
X_train = []
y_train = []
for seq in text_sequences:
    for i in range(len(seq)-1):
        X_train.append(seq[i])
        y_train.append(seq[i+1])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [62]:
# 5. Улучшенная модель GPT для генерации SCP описаний
class SCPTextGenerator:
    def __init__(self, vocab_size, embedding_dim=32, hidden_dim=64):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Инициализация весов с учетом Xavier/Glorot инициализации
        limit_embed = np.sqrt(6 / (vocab_size + embedding_dim))
        self.W_embed = np.random.uniform(-limit_embed, limit_embed, (vocab_size, embedding_dim))

        limit_hidden = np.sqrt(6 / (embedding_dim + hidden_dim))
        self.W_hidden = np.random.uniform(-limit_hidden, limit_hidden, (embedding_dim, hidden_dim))

        limit_out = np.sqrt(6 / (hidden_dim + vocab_size))
        self.W_output = np.random.uniform(-limit_out, limit_out, (hidden_dim, vocab_size))

        self.b_hidden = np.zeros(hidden_dim)
        self.b_output = np.zeros(vocab_size)

        # Для отслеживания потерь
        self.loss_history = []

    def forward(self, x):
        # Встраивание
        embed = self.W_embed[x]

        # Скрытый слой с ограничением значений
        h = np.tanh(np.dot(embed, self.W_hidden) + self.b_hidden)
        h = np.clip(h, -5, 5)  # Предотвращаем взрыв градиентов

        # Выходной слой
        logits = np.dot(h, self.W_output) + self.b_output
        probs = stable_softmax(logits)
        return probs, h, embed

    def backward(self, x, y, probs, h, embed, learning_rate):
        # Градиенты
        grad_logits = probs.copy()
        grad_logits[range(len(y)), y] -= 1
        grad_logits /= len(y)

        # Градиенты параметров с L2 регуляризацией
        l2_lambda = 0.001

        # Градиенты выходного слоя
        grad_W_output = np.dot(h.T, grad_logits) + l2_lambda * self.W_output
        grad_b_output = np.sum(grad_logits, axis=0)

        # Градиенты скрытого слоя
        grad_h = np.dot(grad_logits, self.W_output.T)
        grad_h_raw = grad_h * (1 - h**2)

        grad_W_hidden = np.dot(embed.T, grad_h_raw) + l2_lambda * self.W_hidden
        grad_b_hidden = np.sum(grad_h_raw, axis=0)

        # Градиенты встраивания
        grad_embed = np.dot(grad_h_raw, self.W_hidden.T)

        # Обновление параметров
        self.W_output -= learning_rate * grad_W_output
        self.b_output -= learning_rate * grad_b_output
        self.W_hidden -= learning_rate * grad_W_hidden
        self.b_hidden -= learning_rate * grad_b_hidden

        # Обновляем только использованные встраивания
        for i, word_idx in enumerate(x):
            self.W_embed[word_idx] -= learning_rate * grad_embed[i]

    def compute_loss(self, probs, y):
        return -np.mean(np.log(probs[range(len(y)), y] + 1e-8))  # Добавляем небольшое значение для стабильности

    def train(self, X, y, epochs=200, learning_rate=0.05, batch_size=8):
        for epoch in range(epochs):
            # Перемешиваем данные каждый эпох
            indices = np.arange(len(X))
            np.random.shuffle(indices)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            total_loss = 0
            correct = 0

            for i in range(0, len(X), batch_size):
                # Получаем мини-пакет
                x_batch = X_shuffled[i:i+batch_size]
                y_batch = y_shuffled[i:i+batch_size]

                # Прямое распространение
                probs, h, embed = self.forward(x_batch)

                # Вычисление потерь и точности
                loss = self.compute_loss(probs, y_batch)
                total_loss += loss * len(x_batch)
                correct += np.sum(np.argmax(probs, axis=1) == y_batch)

                # Обратное распространение
                self.backward(x_batch, y_batch, probs, h, embed, learning_rate)

            # Средние значения за эпоху
            avg_loss = total_loss / len(X)
            accuracy = correct / len(X)
            self.loss_history.append(avg_loss)

            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}")

    def generate_text(self, word_to_idx, idx_to_word, start_word, length=15, temperature=0.7):
        current_word = word_to_idx[start_word.lower()]
        generated = [current_word]

        for _ in range(length-1):
            probs, _, _ = self.forward(np.array([current_word]))
            probs = probs[0]

            if temperature != 1.0:
                probs = np.power(probs, 1.0/temperature)
                probs = probs / np.sum(probs)

            # Убедимся, что нет NaN или inf
            probs = np.nan_to_num(probs, nan=1.0/vocab_size, posinf=1.0/vocab_size, neginf=1.0/vocab_size)
            probs = probs / np.sum(probs)  # Нормализуем на случай ошибок

            try:
                current_word = np.random.choice(len(probs), p=probs)
            except:
                current_word = np.argmax(probs)

            generated.append(current_word)

        return ' '.join([idx_to_word[idx] for idx in generated])

In [63]:
# 6. Создание и обучение модели
print("\nОбучение SCP генерации текста...")
model = SCPTextGenerator(vocab_size, embedding_dim=32, hidden_dim=64)
model.train(X_train, y_train, epochs=200, learning_rate=0.05)


Обучение SCP генерации текста...
Epoch   0 | Loss: 5.2154 | Accuracy: 0.06
Epoch  10 | Loss: 4.7602 | Accuracy: 0.14
Epoch  20 | Loss: 4.5388 | Accuracy: 0.14
Epoch  30 | Loss: 4.2201 | Accuracy: 0.15
Epoch  40 | Loss: 3.9223 | Accuracy: 0.19
Epoch  50 | Loss: 3.6827 | Accuracy: 0.21
Epoch  60 | Loss: 3.4347 | Accuracy: 0.26
Epoch  70 | Loss: 3.1851 | Accuracy: 0.32
Epoch  80 | Loss: 2.9349 | Accuracy: 0.34
Epoch  90 | Loss: 2.6921 | Accuracy: 0.37
Epoch 100 | Loss: 2.4537 | Accuracy: 0.45
Epoch 110 | Loss: 2.2185 | Accuracy: 0.53
Epoch 120 | Loss: 1.9883 | Accuracy: 0.61
Epoch 130 | Loss: 1.7702 | Accuracy: 0.67
Epoch 140 | Loss: 1.5672 | Accuracy: 0.70
Epoch 150 | Loss: 1.3885 | Accuracy: 0.70
Epoch 160 | Loss: 1.2436 | Accuracy: 0.69
Epoch 170 | Loss: 1.1336 | Accuracy: 0.71
Epoch 180 | Loss: 1.0561 | Accuracy: 0.71
Epoch 190 | Loss: 1.0022 | Accuracy: 0.69


In [64]:
# 7. Генерация примеров
print("\nГенерация SCP описания:")
start_words = ["scp-173", "scp-682", "scp-096", "scp-106", "scp-049", "scp-999"]
for word in start_words:
    print(f"\n '{word}':")
    for _ in range(2):
        generated = model.generate_text(word_to_idx, idx_to_word, word, temperature=0.7)
        print(f"- {generated.capitalize()}")


Генерация SCP описания:

 'scp-173':
- Scp-173 - огромное наблюдают, и утаскивает жертв в бесконечную пузырь, порождающий опасных существ в ярость,
- Scp-173 - абель, воин с мечом, который постоянно возрождается - плюшевый медвежонок, который размножается -

 'scp-682':
- Scp-682 - приложение, которое показывает вам существо слизень, который может вылечить болезнь - мутировавшие существа,
- Scp-682 - кукла, которая двигается, которое крайне сложно уничтожить создаёт своих копий существа, имитирующие человеческие

 'scp-096':
- Scp-096 - врач-чумной доктор, который размножается - старик, который впадает в бесконечную темноту уничтожить магазин
- Scp-096 - существо, которое показывает вам существо рептилоидное существо, которое показывает вам существо - хищные

 'scp-106':
- Scp-106 - плюшевый медведь, который может вылечить болезнь в водоёмах космический корабль, который манипулирует информацией
- Scp-106 - паразитическая амфибия, обитающая в бесконечную темноту человеческие голоса пор